## Введение в МО: каким оно бывает и каковы основные компоненты


# 1
Машинное обучение с учителем подразумевает:

`Обучение на прецедентах - объектах и соответствуюших им ответам.`

# 2

`Вещественными признаками и таргетами (ответами)` называют такие, которые принимают значения исключительно из некоторого установленного множества: например, 0 или 1

`Не верно`

Пояснение: называя признак или ответ вещественным, мы ожидаем, что чем больше данных у нас будет, тем больше различных уникальных значений для наших объектов встретятся в этих признаках или таргетах. Простой пример – признак, описывающий вес человека. При определенной точности замера, практически у всех людей он будет отличаться. А совпадения хоть и будут, но не так часто, как, например, для признака “пол человека”.

# 3

В выборке не может быть пропущенных значений. А если они и встречаются, нам необходимо удалить все объекты, у которых хотя бы один признак (или ответ) отсутствует.

`Не обязательно. Мы можем обрабатывать такие проблемные места. Например, если признак имеет временную структуру, можем заполнять пропуски предыдущими значениями.`

# 4

`Категориальные признаки` часто “декодируют”, чтобы получить некоторый числовой эквивалент текстовому описанию объекта. Например, это можно делать с помощью счетчиков. Где процедура описана наиболее правильно и полно?

`Необходимо понять, как категории между собой отличаются с точки зрения своего влияния на таргет. Посчитаем какую-то аггрегацию каждой категории поверх нашего таргета: например, его среднее значение для каждой группы объектов (категории). Заменим категории на соответствующие им значения. `

# Практика
Описание данных:

`id` - ID поездки

`vendor_id` - ID компании, осуществляющей перевозку

`pickup_datetime` - Таймкод начала поездки

`dropoff_datetime` - Таймкод конца поездки

`passenger_count` - Количество пассажиров

`pickup_longitude` - Долгота точки, в которой началась поездка

`pickup_latitude` - Широта точки, в которой началась поездка

`dropoff_longitude` - Долгота точки, в которой закончилась поездка

`dropoff_latitude` - Широта точки, в которой закончилась поездка

`store_and_fwd_flag- Yes/No:` Была ли информация сохранена в памяти транспортного средства из-за потери соединения с сервером

# 5

Давайте проверим, как вы справились с 

- кодировкой бинарных признаков (колонки vendor_id и store_and_fwd_flag)

- Обратите внимание, что у вас при кодировании store_and_fwd_flag 'N' кодируется как 0, а 'Y' как 1

- Выполните необходимые вычисления в ноутбуке. 
- После того, как вы изменили эти колонки, сохраните первые 10 значений полученного промежуточного датафрейма в файл в формате csv с сепаратором ;. Отправьте полученный файл в форму ниже